# Sentiment analysis with Recurrent Neural Networks

(this model was unsuccessful at providing meaningful results)

We used the pytorch module to implement this Neural network by hand

In [4]:
#imports
import timeit
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import re
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
import random
from nltk.tokenize.casual import casual_tokenize

In [5]:
#read in data
df = pd.read_csv('../Downloads/sentiment140.csv', header = None, encoding = "ISO-8859-1")
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text'] #set column names
print(len(df))

#removing neutrals because they are very few
df = df.drop(df[df.target == 2].index)
print(len(df))
print(df.target.unique())

#add column with tweets tokenized
df["words"] = [tuple([word for word in casual_tokenize(row.text) if not re.match('@', 
  word)]) for _, row in df.iterrows()]
df["num_words"] = [len(row.words) for _, row in df.iterrows()]
print(df.head())

1600498
1600359
[4 0]
   target  id                          date     flag      user  \
0       4   3  Mon May 11 03:17:40 UTC 2009  kindle2    tpryan   
1       4   4  Mon May 11 03:18:03 UTC 2009  kindle2    vcu451   
2       4   5  Mon May 11 03:18:54 UTC 2009  kindle2    chadfu   
3       4   6  Mon May 11 03:19:04 UTC 2009  kindle2     SIX15   
4       4   7  Mon May 11 03:21:41 UTC 2009  kindle2  yamarama   

                                                text  \
0  @stellargirl I loooooooovvvvvveee my Kindle2. ...   
1  Reading my kindle2...  Love it... Lee childs i...   
2  Ok, first assesment of the #kindle2 ...it fuck...   
3  @kenburbary You'll love your Kindle2. I've had...   
4  @mikefish  Fair enough. But i have the Kindle2...   

                                               words  num_words  
0  (I, loooooooovvvvvveee, my, Kindle, 2, ., Not,...         23  
1  (Reading, my, kindle, 2, ..., Love, it, ..., L...         14  
2  (Ok, ,, first, assesment, of, the, #kindle2

In [6]:
#split testing and training sets
train = df.sample(frac=.9)
test = df.drop(train.index)
test['prediction'] = np.nan
print(len(train), len(test))
print(test.head())

1440323 160036
    target   id                          date       flag            user  \
32       4  118  Sat May 16 16:19:04 UTC 2009     google          J_Holl   
34       0  120  Sat May 16 16:25:41 UTC 2009        aig        schroncd   
77       4  195  Sun May 24 16:19:04 UTC 2009        50d   justinbettman   
92       0  214  Mon May 25 17:26:50 UTC 2009    insects  AntoineTheReaL   
95       4  217  Mon May 25 17:29:39 UTC 2009  mcdonalds       MamiYessi   

                                                 text  \
32  @phyreman9 Google is always a good place to lo...   
34  US planning to resume the military tribunals a...   
77      Class... The 50d is supposed to come today :)   
92  wish i could catch every mosquito in the world...   
95  omgg i ohhdee want mcdonalds damn i wonder if ...   

                                                words  num_words  prediction  
32  (Google, is, always, a, good, place, to, look,...         22         NaN  
34  (US, planning, to, resu

We tried two different methods of word embedding.

In [8]:
#word embedding method 1
#word to index embedding
#vocab dictionary {word: index} 
word_to_ix = {}
ix = 0
for _, tweet in train.iterrows():
   for word in tweet.words:
       if word not in word_to_ix:
           word_to_ix[word] = ix
           ix += 1
word_to_ix["__dummy__"] = ix
#use word to index dictionary to create word embeddings
embeds = nn.Embedding(len(word_to_ix), 100)  # n words in vocab, 150 dimensional embeddings
print(embeds)
len(word_to_ix)

Embedding(466243, 100)


466243

In [9]:
%%time
#word embedding method 2
#word-to-vector model
W2V_SIZE = 150
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10
documents = train.words.tolist()
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)
w2v_model.build_vocab(documents)
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

ok


(501414228, 681274048)

Then we defined the model and some functions to embed and back propagate

In [10]:
#create a recurrent neural network model using pytorch
class model(nn.Module):
    def __init__(self, num_input, num_hidden, num_output):
        super(model, self).__init__()
        self.rnn = nn.GRU(num_input, num_hidden)
        self.linear = nn.Linear(num_hidden, num_output)
        self.activate = nn.Sigmoid()
    def forward(self, x):
        _, last_hidden = self.rnn(x)
        z = self.linear(last_hidden)
        y_ = self.activate(z)
        return y_[0]

def embed(batch):
    """converts batch of tweets into word embeddings"""
    x, y = [], []
    for _, tweet in batch.iterrows():
        embedding = []
        for word in tweet.words:
#using word embedding method 2
            if word in w2v_model:
                embedding.append(w2v_model[word])
            else:
                embedding.append([random.uniform(-4, 4) for i in range(150)])
        x.append(embedding)
##alternate method using word embedding method 1
#            if word in word_to_ix:
#                lookup_tensor = torch.tensor([word_to_ix[word]], dtype=torch.long)
#            else:
#                lookup_tensor = [random.random 
#            word_embed = embeds(lookup_tensor).tolist()[0]
#            embedding.append(word_embed)
#        x.append(embedding)
        y.append([tweet.target != 0])
    x = torch.FloatTensor(x) #(len(tweet), 150, 150)
    x = x.transpose(0, 1)
    y = torch.FloatTensor(y)
    return x, y

def back_propagate(x, y, m):
    """trains the rnn model"""
    loss_function = nn.BCELoss()
    optimizer = optim.SGD(m.parameters(), lr=0.01)
    for epoch in range(1000):
        m.zero_grad()
        y_ = m(x) # HK
        loss = loss_function(y_, y) # HK
        loss.backward()
        optimizer.step()
    return m

Then we train the model

In [ ]:
%%time
m = model(150, 100, 1) #initiate model

#train the model in batches of up to 150 tweets at a time
begin, end = 1, 101
for length in range(begin, end): #I ommitted all the tweets with >100 words because they were mostly gibberish
    megabatch = train[train.num_words == length]
    if len(megabatch) == 0:
        continue

    while len(megabatch) > 2000:
        batch = megabatch.sample(2000)
        megabatch = megabatch.drop(batch.index)
        x, y = embed(batch)
        m = back_propagate(x, y, m)
    x, y = embed(megabatch)
    m = back_propagate(x, y, m)

And evaluate the results using the test data

In [ ]:
accurate = 0
for length in range(1, 101): #I ommitted all the tweets with >100 words because they were mostly gibberish
    megabatch = test[test.num_words == length]
    if len(megabatch) == 0:
        continue
    x, y = embed(megabatch)
    y_ = m(x)
    for i in range(len(y)):
        prediction = round(float(y_[i]))
        #test.loc[megabatch.index[i], 'prediction'] = prediction
        if y[i] == prediction:
            accurate += 1
    break
print("Accuracy: ", accurate/len(test))